In [1]:
%pylab inline
from classy import *

Populating the interactive namespace from numpy and matplotlib
Version  1.0.1
Version:  0.0.21


In [2]:
from NumPyNet.network import Network
from NumPyNet.layers.connected_layer import Connected_layer
from NumPyNet.layers.cost_layer import Cost_layer
from NumPyNet.optimizer import Adam,SGD
from NumPyNet.utils import to_categorical
from NumPyNet.utils import from_categorical
from NumPyNet.metrics import mean_accuracy_score

In [3]:
data=load_excel('data/iris.xls')
data_train,data_test=split(data,test_size=0.2)

iris.data 151 5
150 vectors of length 4
Feature names: 'petal length in cm', 'petal width in cm', 'sepal length in cm', 'sepal width in cm'
Target values given.
Target names: 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'
Mean:  [3.75866667 1.19866667 5.84333333 3.054     ]
Median:  [4.35 1.3  5.8  3.  ]
Stddev:  [1.75852918 0.76061262 0.82530129 0.43214658]
Original vector shape:  (150, 4)
Train vector shape:  (120, 4)
Test vector shape:  (30, 4)


In [4]:
X=data.vectors
Y=data.targets

In [5]:
X.shape

(150, 4)

In [6]:
Y.shape

(150,)

In [7]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
one_hot_y=to_categorical(Y)
one_hot_y[40:60,:]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [9]:
def accuracy (y_true, y_pred):
    '''
    Temporary metrics to overcome "from_categorical" missing in standard metrics
    '''
    from NumPyNet.metrics import mean_accuracy_score
    truth = from_categorical(y_true)
    predicted = from_categorical(y_pred)
    return mean_accuracy_score(truth, predicted)


In [10]:
X_train=data_train.vectors
y_train=data_train.targets
one_hot_y_train=to_categorical(y_train)

X_test=data_test.vectors
y_test=data_test.targets
one_hot_y_test=to_categorical(y_test)

In [11]:
# Reshape the data according to a 4D tensor
num_samples, size = X_test.shape
num_samples, size

num_classes=one_hot_y_test.shape[1]

X_test = X_test.reshape(num_samples, 1, 1, size)
one_hot_y_test = one_hot_y_test.reshape(num_samples,1,1,-1)

X_test.shape,one_hot_y_test.shape

((30, 1, 1, 4), (30, 1, 1, 3))

In [12]:
# Reshape the data according to a 4D tensor
num_samples, size = X_train.shape
num_samples, size

num_classes=one_hot_y_train.shape[1]

X_train = X_train.reshape(num_samples, 1, 1, size)
one_hot_y_train = one_hot_y_train.reshape(num_samples,1,1,-1)

X_train.shape,one_hot_y_train.shape

((120, 1, 1, 4), (120, 1, 1, 3))

In [13]:
batch=len(y_train)
step = batch

In [14]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=6, activation='tanh'))
model.add(Connected_layer(outputs=num_classes, activation='tanh'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])

In [15]:
model.summary()

layer       filters  size              input                output
   0 input                   120 x   1 x   1 x   4   ->   120 x   1 x   1 x   4
   1 connected               120 x   1 x   1 x   4   ->   120 x   6
   2 connected               120 x   1 x   1 x   6   ->   120 x   3
   3 cost                    120 x   1 x   1 x   3   ->   120 x   1 x   1 x   3


In [16]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.010 accuracy: 0.300

Epoch 2/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.010 accuracy: 0.300

Epoch 3/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.010 accuracy: 0.300

Epoch 4/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.010 accuracy: 0.300

Epoch 5/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.009 accuracy: 0.300

Epoch 6/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.009 accuracy: 0.300

Epoch 7/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.009 accuracy: 0.300

Epoch 8/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.009 accuracy: 0.300

Epoch 9/1000
120/120 |██████████████████████████████████

  2%|▏         | 23/1000 [00:00<00:04, 225.10it/s]

Epoch 24/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.008 accuracy: 0.300

Epoch 25/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.008 accuracy: 0.300

Epoch 26/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.008 accuracy: 0.300

Epoch 27/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.008 accuracy: 0.300

Epoch 28/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.008 accuracy: 0.300

Epoch 29/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.008 accuracy: 0.300

Epoch 30/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.007 accuracy: 0.300

Epoch 31/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.007 accuracy: 0.300

Epoch 32/1000
120/120 |█████████████████████████

  5%|▍         | 49/1000 [00:00<00:04, 232.02it/s]

...

Epoch 963/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 964/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 965/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 966/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 967/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 968/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 969/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 970/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.900

Epoch 971/1000
120/120 |████████████████

 99%|█████████▊| 987/1000 [00:04<00:00, 237.42it/s]

Epoch 988/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.917

Epoch 989/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.917

Epoch 990/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.917

Epoch 991/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.917

Epoch 992/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.917

Epoch 993/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.925

Epoch 994/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.933

Epoch 995/1000
120/120 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001 accuracy: 0.933

Epoch 996/1000
120/120 |████████████████

100%|██████████| 1000/1000 [00:04<00:00, 235.24it/s]

Training on 1000 epochs took 4.3 sec


In [56]:
model.fit?

Signature: model.fit(X, y, max_iter=100, shuffle=True, verbose=True)
Docstring:     
File:      ~/opt/anaconda3/envs/py3/lib/python3.7/site-packages/NumPyNet-1.0.0-py3.7.egg/NumPyNet/network.py
Type:      method


In [57]:
model.batch

120

In [3]:
model.batch=len(X_test)
loss, out = model.evaluate(X=X_test, truth=one_hot_y_test, verbose=True)

NameError: name 'X_test' is not defined

In [59]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))


Loss Score: 0.002
Accuracy Score: 0.967


In [2]:
L1=model._net[1]

NameError: name 'model' is not defined

In [63]:
model._net[2].weights.shape

(6, 3)

In [61]:
model._net

In [29]:
truth.ravel(),predicted.ravel()

(array([1, 1, 1, 1, 0, 0, 1, 2, 1, 2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 1, 2, 1,
        0, 1, 2, 0, 2, 0, 0, 2]),
 array([1, 1, 1, 1, 0, 0, 1, 2, 1, 2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 1, 2, 1,
        0, 1, 2, 0, 2, 0, 0, 2]))

## Images

In [64]:
images=image.load_images('data/digits')

[9]: 180 files found
[0]: 178 files found
[7]: 179 files found
[6]: 181 files found
[1]: 182 files found
[8]: 174 files found
[4]: 181 files found
[3]: 183 files found
[2]: 177 files found
[5]: 182 files found


In [65]:
data=image.images_to_vectors(images)

1797 vectors of length 64
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p59', 'p60', 'p61', 'p62', 'p63'  (64 features)
Target values given.
Target names: '9', '0', '7', '6', '1', '8', '4', '3', '2', '5'
Mean:  [0.00000000e+00 4.71341124e+00 8.25158598e+01 1.88401224e+02
 1.88589872e+02 9.17840846e+01 2.15581525e+01 2.04785754e+00
 8.51419032e-02 3.14913745e+01 1.65203116e+02 1.90677240e+02
 1.63504174e+02 1.30023372e+02 2.92036728e+01 1.70005565e+00
 4.22927101e-02 4.11185309e+01 1.57575960e+02 1.11080690e+02
 1.12814691e+02 1.24173066e+02 2.82609905e+01 7.82971619e-01
 1.66944908e-02 3.90011130e+01 1.44642181e+02 1.40286589e+02
 1.58007791e+02 1.20022816e+02 3.66889260e+01 3.33889816e-02
 0.00000000e+00 3.70083472e+01 1.21928770e+02 1.44347245e+02
 1.63978854e+02 1.39081803e+02 4.60222593e+01 0.00000000e+00
 1.37451308e-01 2.49860879e+01 1.09460768e+02 1.14961046e+02
 1.21993879e+02 1.30974958e+02 5.47762938e+01 4.24040067e-01
 1.13522538e-01 1.10378408e+01 1.19312187e+02 1.

In [66]:
data.vectors/=255
summary(data)

1797 vectors of length 64
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p59', 'p60', 'p61', 'p62', 'p63'  (64 features)
Target values given.
Target names: '9', '0', '7', '6', '1', '8', '4', '3', '2', '5'
Mean:  [0.00000000e+00 1.84839657e-02 3.23591607e-01 7.38828330e-01
 7.39568126e-01 3.59937587e-01 8.45417744e-02 8.03081388e-03
 3.33889816e-04 1.23495586e-01 6.47855358e-01 7.47753882e-01
 6.41192838e-01 5.09895578e-01 1.14524207e-01 6.66688490e-03
 1.65853765e-04 1.61249141e-01 6.17944941e-01 4.35610549e-01
 4.42410554e-01 4.86953201e-01 1.10827414e-01 3.07047694e-03
 6.54685914e-05 1.52945541e-01 5.67224241e-01 5.50143485e-01
 6.19638395e-01 4.70677709e-01 1.43878141e-01 1.30937183e-04
 0.00000000e+00 1.45130774e-01 4.78152040e-01 5.66067629e-01
 6.43054328e-01 5.45418835e-01 1.80479448e-01 0.00000000e+00
 5.39024736e-04 9.79846585e-02 4.29257914e-01 4.50827632e-01
 4.78407367e-01 5.13627287e-01 2.14808995e-01 1.66290222e-03
 4.45186422e-04 4.32856504e-02 4.67890929e-01 5.

In [67]:
data_train,data_test=split(data)

Original vector shape:  (1797, 64)
Train vector shape:  (1347, 64)
Test vector shape:  (450, 64)


In [68]:
X_train=data_train.vectors
y_train=data_train.targets
one_hot_y_train=to_categorical(y_train)

X_test=data_test.vectors
y_test=data_test.targets
one_hot_y_test=to_categorical(y_test)

In [69]:
one_hot_y_test[:10,:]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [70]:
# Reshape the data according to a 4D tensor
num_samples, size = X_test.shape
num_samples, size

num_classes=one_hot_y_test.shape[1]

X_test = X_test.reshape(num_samples, 1, 1, size)
one_hot_y_test = one_hot_y_test.reshape(num_samples,1,1,-1)

X_test.shape,one_hot_y_test.shape

((450, 1, 1, 64), (450, 1, 1, 10))

In [71]:
# Reshape the data according to a 4D tensor
num_samples, size = X_train.shape
num_samples, size

num_classes=one_hot_y_train.shape[1]

X_train = X_train.reshape(num_samples, 1, 1, size)
one_hot_y_train = one_hot_y_train.reshape(num_samples,1,1,-1)

X_train.shape,one_hot_y_train.shape

((1347, 1, 1, 64), (1347, 1, 1, 10))

In [72]:
batch=len(y_train)
step = batch

In [73]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=num_classes, activation='logistic'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

layer       filters  size              input                output
   0 input                  1347 x   1 x   1 x  64   ->  1347 x   1 x   1 x  64
   1 connected              1347 x   1 x   1 x  64   ->  1347 x  10
   2 cost                   1347 x   1 x   1 x  10   ->  1347 x   1 x   1 x  10


In [74]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=10000,verbose=False)

100%|██████████| 10000/10000 [00:18<00:00, 534.70it/s]


In [75]:
model.batch=len(X_test)
loss, out = model.evaluate(X=X_test, truth=one_hot_y_test, verbose=True)

450/450 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.000
Prediction on 450 samples took 0.0 sec


In [76]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))


Loss Score: 0.000
Accuracy Score: 0.969


## convo net

In [77]:
from NumPyNet.layers.convolutional_layer import Convolutional_layer
from NumPyNet.layers.maxpool_layer import Maxpool_layer
from NumPyNet.layers.softmax_layer import Softmax_layer
from NumPyNet.layers.batchnorm_layer import BatchNorm_layer


In [80]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
digits = datasets.load_digits()
X, y = digits.images, digits.target

In [81]:
# add channels to images
X = np.asarray([np.dstack((x, x, x)) for x in X])
X = X.transpose(0, 2, 3, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                  test_size=.33,
                                                  random_state=42)

In [78]:
batch=128

model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Convolutional_layer(size=3, filters=32, stride=1, pad=True, activation='Relu'))
model.add(BatchNorm_layer())
model.add(Maxpool_layer(size=2, stride=1, padding=True))
model.add(Connected_layer(outputs=100, activation='Relu'))
model.add(BatchNorm_layer())
model.add(Connected_layer(outputs=num_classes, activation='Linear'))
model.add(Softmax_layer(spatial=True, groups=1, temperature=1.))
model.add(Cost_layer(cost_type=cost_type.mse))

# model.compile(optimizer=SGD(lr=0.01, decay=0., lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

/Users/bblais/opt/anaconda3/envs/py3/lib/python3.7/site-packages/NumPyNet-1.0.0-py3.7.egg/NumPyNet/layers/connected_layer.py:73: RuntimeWarning: divide by zero encountered in true_divide
  scale = np.sqrt(2. / self.inputs)


OverflowError: Range exceeds valid bounds

In [ ]:
X_train.shape[1:]